<img src='etc/콜백함수.png' width=1000 height= 600>

In [1]:
#1.OS환경만들기  
import os  # 탐색기 모듈. os.path
os.environ['KERAS_BACKEND']  ='tensorflow'
kerasBKED  = os.environ['KERAS_BACKEND']
print(kerasBKED)

tensorflow


In [2]:
# 2.모듈 추가.  
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, InputLayer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
#3.작업 디렉토리 생성
saveDir='./cifar10/'
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)

In [4]:
#4. 데이터 로드  
batch_size  = 32
num_classes =10
epochs  =10 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)
print(x_train.shape[0])
print(x_test.shape[0])

y_train  =tf. keras.utils.to_categorical(y_train, num_classes)
y_test  = tf.keras.utils.to_categorical(y_test, num_classes)

x_train =x_train.astype('float32')
x_test  = x_test.astype('float32')

x_train /= 255

(50000, 32, 32, 3)
50000
10000


In [5]:
#5 모형만들기  , 모델 만들기  
model  = Sequential() 
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [6]:
#6. 컴파일 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
#7. 콜백함수 
es_cb = EarlyStopping(monitor='val_loss' , patience =2, verbose =1, mode='auto')

file_path= os.path.join(saveDir,'Cifar10_convert.{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5' )
cp_cb = ModelCheckpoint(filepath =file_path , monitor ='val_loss',verbose=1, save_best_only =True, mode ='auto' )

In [8]:
#8. 학습실행  
history = model.fit(x_train, y_train, 
                    batch_size = batch_size, 
                    epochs= epochs , 
                    validation_data =  (x_test, y_test),
                    callbacks =[es_cb ,  cp_cb] , 
                    shuffle= True)

Epoch 1/10


1563/1563 [==============================] - ETA: 0s - loss: 1.4354 - accuracy: 0.4768
Epoch 1: val_loss improved from inf to 169.70554, saving model to ./cifar10\Cifar10_convert.01-1.44-169.71.hdf5
1563/1563 [==============================] - 60s 36ms/step - loss: 1.4354 - accuracy: 0.4768 - val_loss: 169.7055 - val_accuracy: 0.3931
Epoch 2/10
   3/1563 [..............................] - ETA: 45s - loss: 1.2128 - accuracy: 0.5729

C:\Users\taeeon.kim\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1561/1563 [============================>.] - ETA: 0s - loss: 1.0293 - accuracy: 0.6362
Epoch 2: val_loss did not improve from 169.70554
1563/1563 [==============================] - 56s 36ms/step - loss: 1.0293 - accuracy: 0.6361 - val_loss: 244.9412 - val_accuracy: 0.3570
Epoch 3/10
1561/1563 [============================>.] - ETA: 0s - loss: 0.8762 - accuracy: 0.6924
Epoch 3: val_loss improved from 169.70554 to 135.83369, saving model to ./cifar10\Cifar10_convert.03-0.88-135.83.hdf5
1563/1563 [==============================] - 52s 33ms/step - loss: 0.8763 - accuracy: 0.6924 - val_loss: 135.8337 - val_accuracy: 0.4930
Epoch 4/10
1562/1563 [============================>.] - ETA: 0s - loss: 0.7759 - accuracy: 0.7261
Epoch 4: val_loss improved from 135.83369 to 99.96165, saving model to ./cifar10\Cifar10_convert.04-0.78-99.96.hdf5
1563/1563 [==============================] - 49s 32ms/step - loss: 0.7759 - accuracy: 0.7261 - val_loss: 99.9616 - val_accuracy: 0.5746
Epoch 5/10
1561/1563 [==

In [9]:
#!pip install h5py

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import h5py
f = h5py.File('./cifar10/Cifar10_convert.04-0.78-99.96.hdf5', 'r')
list(f.keys())

['model_weights', 'optimizer_weights']

In [13]:
import pandas as pd
df =pd.DataFrame(f)
df

,0
0,model_weights
1,optimizer_weights


In [14]:
model.load_weights('./cifar10/Cifar10_convert.04-0.78-99.96.hdf5')

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                        

In [23]:
for layer in model.layers:
        weights = layer.get_weights()
        if weights:
            print(f'{layer.name} shape {weights[0].shape}')
            print(f'{layer.name} min {weights[0].min()}')
            print(f'{layer.name} max {weights[0].max()}')
            
            if len(weights) > 1:
                print(f' 1 : {layer.name} bias shape {weights[1].shape}')
                print(f' 1 : {layer.name} bias min {weights[1].min()}')
                print(f' 1 : {layer.name} bias max {weights[1].max()}')
                print(f' 1 : {layer.name} bias mean {weights[1].mean()}')

conv2d shape (3, 3, 3, 32)
conv2d min -0.29388654232025146
conv2d max 0.25933653116226196
 1 : conv2d bias shape (32,)
 1 : conv2d bias min -0.1412496268749237
 1 : conv2d bias max 0.09859113395214081
 1 : conv2d bias mean -0.0016726329922676086
conv2d_1 shape (3, 3, 32, 32)
conv2d_1 min -0.4659684896469116
conv2d_1 max 0.34798142313957214
 1 : conv2d_1 bias shape (32,)
 1 : conv2d_1 bias min -0.05662478134036064
 1 : conv2d_1 bias max 0.06535575538873672
 1 : conv2d_1 bias mean -0.007831165567040443
conv2d_2 shape (3, 3, 32, 64)
conv2d_2 min -0.627456545829773
conv2d_2 max 0.36283427476882935
 1 : conv2d_2 bias shape (64,)
 1 : conv2d_2 bias min -0.13494592905044556
 1 : conv2d_2 bias max 0.16048304736614227
 1 : conv2d_2 bias mean 0.00380896870046854
conv2d_3 shape (3, 3, 64, 64)
conv2d_3 min -0.5458937287330627
conv2d_3 max 0.37517398595809937
 1 : conv2d_3 bias shape (64,)
 1 : conv2d_3 bias min -0.07902579754590988
 1 : conv2d_3 bias max 0.15205074846744537
 1 : conv2d_3 bias mean

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow

In [24]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
model.fit(x_train, y_train, epochs=2, callbacks=[tensorboard_callback])
# Then run the tensorboard command to view the visualizations.

Epoch 1/2
1563/1563 [==============================] - 48s 30ms/step - loss: 0.6999 - accuracy: 0.7547
Epoch 2/2
1563/1563 [==============================] - 44s 28ms/step - loss: 0.6479 - accuracy: 0.7717
